In [ ]:
!ls requirements.txt
!pip install -r requirements.txt
import tensorflow_io as tfio
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from preprocessing import LABELS

requirements.txt
  Using cached psutil-5.9.2-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (281 kB)
  Using cached MarkupSafe-2.1.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: psutil
    Found existing installation: psutil 5.9.3
    Not uninstalling psutil at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'psutil'. No files were found to uninstall.
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
2022-12-20 16:09:06.105707: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load 

In [ ]:
go_stop_train_ds = tf.data.Dataset.list_files(['msc-train/go*', 'msc-train/stop*'])
go_stop_val_ds = tf.data.Dataset.list_files(['msc-val/go*', 'msc-val/stop*'])
go_stop_test_ds = tf.data.Dataset.list_files(['msc-test/go*', 'msc-test/stop*'])

2022-12-20 16:09:10.918474: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-20 16:09:10.918508: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-20 16:09:10.918530: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-59bac526-4289-456d-8cb9-47b42ab71644): /proc/driver/nvidia/version does not exist
2022-12-20 16:09:10.918891: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import os
import numpy as np
import random
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed)
np.random.seed(seed)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=[32, 32, 1]),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=[1, 1], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=[1, 1], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
])

In [ ]:
from preprocessing import get_audio_and_label
from preprocessing import get_spectrogram
from preprocessing import get_log_mel_spectrogram
from functools import partial
TRAINING_ARGS = {
    'batch_size': 15,
    'epochs': 10,
    'initial_learning_rate': 0.01,
    'end_learning_rate': 1.e-5,
}
batch_size = TRAINING_ARGS['batch_size']
initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']
epoch = TRAINING_ARGS['epochs']
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=end_learning_rate,
    decay_steps= len(go_stop_train_ds) * epoch,
)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
for mlf in [20, 50, 80]:
    for mul in [2000, 5000, 8000]:
        for bin in [10, 20, 30, 40]:
            for down_s in [4000, 8000, 16000]:
                for frame_l in [0.01, 0.02, 0.04, 0.05]:
                    for overlap_perc in [0.25, 0.50, 0.75, 1]:
                        overlap = frame_l*(1-overlap_perc)
                        SPECTROGRAM_ARGS = {
                            'downsampling_rate': down_s,
                            'frame_length_in_s': frame_l,
                            'frame_step_in_s': overlap
                        }
                        MEL_LOG_ARGS = {
                            'downsampling_rate': down_s,
                            'frame_length_in_s': frame_l,
                            'frame_step_in_s': frame_l,
                            'num_mel_bins': bin,
                            'lower_frequency': mlf,
                            'upper_frequency': mul,
                        }

                        def get_spectrogram_and_label(filename, downsampling_rate, frame_length_in_s, frame_step_in_s):
                            x, _, label = get_spectrogram(filename, downsampling_rate, frame_length_in_s, frame_step_in_s)
                            return x, label
                        get_frozen_spectrogram = partial(get_spectrogram_and_label, **SPECTROGRAM_ARGS)
                        get_frozen_log_mel_spectrogram = partial(get_log_mel_spectrogram, **MEL_LOG_ARGS)
                        SHAPE = [49, 40]

                        def preprocess_with_resized_mel(filename):
                            signal, label = get_frozen_log_mel_spectrogram(filename)
                            signal.set_shape(SHAPE) 
                            signal = tf.expand_dims(signal, -1)
                            signal = tf.image.resize(signal, [32, 32])
                            label_id = tf.argmax(label == LABELS)
                            return signal, label_id

            
                        initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
                        end_learning_rate = TRAINING_ARGS['end_learning_rate']
                        epoch = TRAINING_ARGS['epochs']
                        train_resized_ds = go_stop_train_ds.map(preprocess_with_resized_mel).batch(batch_size)
                        val_ds = go_stop_val_ds.map(preprocess_with_resized_mel).batch(batch_size)
                        test_ds = go_stop_test_ds.map(preprocess_with_resized_mel).batch(batch_size)
                        loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)

                        linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
                            initial_learning_rate=initial_learning_rate,
                            end_learning_rate=end_learning_rate,
                            decay_steps= len(go_stop_train_ds) * epoch,
                        )
                        optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
                        metrics = [tf.metrics.SparseCategoricalAccuracy()]
                        model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
                        history = model.fit(train_resized_ds, epochs=epoch, validation_data=val_ds)
                        test_loss, test_accuracy = model.evaluate(test_ds)
                        print("test_accuracy", test_accuracy)
                        if test_accuracy > 0.80:
                            MODEL_NAME = test_accuracy
                            saved_model_dir = f'saved_models/{str(MODEL_NAME)}'
                            if not os.path.exists(saved_model_dir):
                                os.makedirs(saved_model_dir)
                            model.save(saved_model_dir)


2022-12-20 16:09:26.335058: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2022-12-20 16:09:26.335277: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 AVX512F FMA
2022-12-20 16:09:26.718118: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2022-12-20 16:09:26.719597: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2022-12-20 16:09:26.719804: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2022-12-20 16:09:26.975641: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2022-12-20 16:09:26.976933: W tensorflow/core/framework/op_kernel.cc:1745] O

KeyboardInterrupt: 

In [ ]:
#accordingly to the best accuracy obtained we select the hyperparameters
#to use in the following part to fine tune and otmize the model

In [ ]:
TRAINING_ARGS = {
    'batch_size': 15,
    'epochs': 10,
    'initial_learning_rate': 0.01,
    'end_learning_rate': 1.e-5,
}
batch_size = TRAINING_ARGS['batch_size']
initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']
epoch = TRAINING_ARGS['epochs']

In [ ]:
from time import time

good_param = list()
timestamp = time()

SPECTROGRAM_ARGS = {
    'downsampling_rate': 16000,
    'frame_length_in_s': 0.05,
    'frame_step_in_s': 0.025
}

MEL_LOG_ARGS = {
    'downsampling_rate': 16000,
    'frame_length_in_s': 0.05,
    'frame_step_in_s': 0.025,
    'num_mel_bins': 30,
    'lower_frequency': 20,
    'upper_frequency': 8000, 
}


def get_spectrogram_and_label(filename, downsampling_rate, frame_length_in_s, frame_step_in_s):
    x, _, label = get_spectrogram(filename, downsampling_rate, frame_length_in_s, frame_step_in_s)
    return x, label
get_frozen_spectrogram = partial(get_spectrogram_and_label, **SPECTROGRAM_ARGS)
get_frozen_log_mel_spectrogram = partial(get_log_mel_spectrogram, **MEL_LOG_ARGS)
SHAPE = [49, 40]

def preprocess_with_resized_mel(filename):
    signal, label = get_frozen_log_mel_spectrogram(filename)
    signal.set_shape(SHAPE) 
    signal = tf.expand_dims(signal, -1)
    signal = tf.image.resize(signal, [32, 32])
    label_id = tf.argmax(label == LABELS)
    return signal, label_id

initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']
epoch = TRAINING_ARGS['epochs']
train_resized_ds = go_stop_train_ds.map(preprocess_with_resized_mel).batch(batch_size)
val_ds = go_stop_val_ds.map(preprocess_with_resized_mel).batch(batch_size)
test_ds = go_stop_test_ds.map(preprocess_with_resized_mel).batch(batch_size)


prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
begin_step = int(len(train_resized_ds) * epoch * 0.2)
end_step = int(len(train_resized_ds) * epoch)
alpha_list = [1]
final_sparsity_list =[0.70, 0.80, 0.85, 0.95]

for alpha in alpha_list:
    for final_sparsity in final_sparsity_list:
      
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=[32, 32, 1]),
            tf.keras.layers.Conv2D(filters=48 * alpha, kernel_size=[3, 3], strides=[2, 2],
                use_bias=False, padding='valid'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
                use_bias=False, padding='same'),
            tf.keras.layers.Conv2D(filters=48 * alpha, kernel_size=[3, 3], strides=[2, 2],
                use_bias=False, padding='valid'),
            tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
                use_bias=False, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(filters=48 * alpha, kernel_size=[1, 1], strides=[1, 1],
                use_bias=False, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
                use_bias=False, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(units=len(LABELS)),
            tf.keras.layers.Softmax()
        ])

    
        #########################################PRUNING#############################################

        pruning_params = {
            'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
                initial_sparsity=0.20,
                final_sparsity=final_sparsity,
                begin_step=begin_step,
                end_step=end_step,
            )
        }
        model_for_pruning = prune_low_magnitude(model, **pruning_params)

        #########################################DEFINING MODEL#############################################

        loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
        linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
            initial_learning_rate=initial_learning_rate,
            end_learning_rate=end_learning_rate,
            decay_steps= len(go_stop_train_ds) * epoch,
        )
        optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
        metrics = [tf.metrics.SparseCategoricalAccuracy()]
        callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]
        model_for_pruning.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        history = model_for_pruning.fit(train_resized_ds, epochs=epoch, validation_data=val_ds, callbacks=callbacks, verbose = 2)
        test_loss, test_accuracy = model_for_pruning.evaluate(test_ds)
        print("test_accuracy", test_accuracy)
        print("test_loss", test_loss)
        print(alpha, final_sparsity)
        if test_accuracy > 0.97:
            model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
            #save the model with the accuracy as name
            saved_model_dir = f'saved_models_fine_tuning/{str(test_accuracy)}'
            if not os.path.exists(saved_model_dir):
                os.makedirs(saved_model_dir)
            model_for_export.save(saved_model_dir)

        

2022-12-20 16:14:16.601646: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2022-12-20 16:14:16.603237: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2022-12-20 16:14:16.603497: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2022-12-20 16:14:16.876534: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2022-12-20 16:14:16.877994: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2022-12-20 16:14:16.878231: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2022-12-20 16:14:17.140264: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed 

KeyboardInterrupt: 

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(f'./saved_models_fine_tuning/{str(test_accuracy)}')
tflite_model = converter.convert()
tflite_models_dir = './tflite_models_fine_tuning'

if not os.path.exists(tflite_models_dir):
    os.makedirs(tflite_models_dir)

tflite_model_name = os.path.join(tflite_models_dir, f'{str(test_accuracy)}.tflite')
with open(tflite_model_name, 'wb') as fp:
    fp.write(tflite_model)

import zipfile
not_pruned_tflite = os.path.join(tflite_models_dir, f'{test_accuracy}.tflite')
with zipfile.ZipFile(f'{not_pruned_tflite}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(not_pruned_tflite)    
tflite_size = os.path.getsize(tflite_model_name) / 1024.0
zipped_size = os.path.getsize(f'{tflite_model_name}.zip') / 1024.0

print(f'Original tflite size (not pruned model): {tflite_size:.3f} KB')
print(f'Zipped tflite size (pruned model): {zipped_size:.3f} KB')

Original tflite size (not pruned model): 103.746 KB
Zipped tflite size (pruned model): 44.480 KB
2022-12-20 16:24:28.053173: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-12-20 16:24:28.053215: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-12-20 16:24:28.053924: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: ./saved_models_fine_tuning/0.9649999737739563
2022-12-20 16:24:28.056869: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-12-20 16:24:28.056893: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: ./saved_models_fine_tuning/0.9649999737739563
2022-12-20 16:24:28.063772: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-12-20 16:24:28.101971: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: ./

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=59bac526-4289-456d-8cb9-47b42ab71644' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>